In [1]:
from keplergl import KeplerGl
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, LineString

In [2]:
stops = pd.read_csv('./STM GTFS/stops.txt', sep=',')
shapes = pd.read_csv('./STM GTFS/shapes.txt', sep=',')
routes = pd.read_csv('./STM GTFS/routes.txt', sep=',')
trips = pd.read_csv('./STM GTFS/trips.txt', sep=',')
stop_times = pd.read_csv('./STM GTFS/stop_times.txt', sep=',')

In [3]:
shapes_gpd = gpd.GeoDataFrame(shapes, geometry=gpd.points_from_xy(shapes.shape_pt_lon, shapes.shape_pt_lat), crs='EPSG:4326')
shapes_gpd = shapes_gpd.groupby('shape_id')['geometry'].apply(lambda x: LineString(x.tolist()) if x.size > 1 else x.tolist()).reset_index()

In [26]:
route_number = 107

In [27]:
trip = trips[trips['route_id'] == routes.query('route_short_name == @route_number')['route_id'].values[0]]
stops_in_trip = stop_times[stop_times['trip_id'].isin(trip['trip_id'])]
shape_trip = shapes_gpd[shapes_gpd['shape_id'].isin(trip['shape_id'])]
stops_in_trip_details = stops[stops['stop_code'].isin(stops_in_trip['stop_id'])]

In [28]:
stops_in_trip_details = stops_in_trip_details.merge(stops_in_trip, left_on='stop_code', right_on='stop_id').drop_duplicates(subset=['stop_code'])

In [29]:
stops_in_trip_gpd = gpd.GeoDataFrame(
    stops_in_trip_details,
    geometry=gpd.points_from_xy(
        stops_in_trip_details.stop_lon, stops_in_trip_details.stop_lat
    ),
    crs="EPSG:4326",
)

In [35]:
stops_in_trip_gpd[['trip_id']] = stops_in_trip_gpd[['trip_id']].astype(str)

In [15]:
# with open('./config.txt', 'w') as f:
#     f.write(str(map.config))

In [30]:
with open('./config.txt', 'r') as f:
    config = f.read()

config = dict(eval(config))    

In [36]:
map = KeplerGl(height=700, config=config)

map.add_data(
    data=stops_in_trip_gpd[["stop_code", "stop_name", "geometry", "trip_id"]], name="stops_in_trip"
)

map.add_data(data=shape_trip[['geometry']], name="shape_trip")

map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'dlwfdmh', 'type': …